# CS246 - Colab 4
## Collaborative Filtering

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 67kB/s 
     |████████████████████████████████| 204kB 48.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=1b80ffce907976ffd1bd7e22b0aa282d2c4f70dec70bb6f512fee8736f49e4ee
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 36.5 MB of archives.
After this 

Now we authenticate a Google Drive client to download the filea we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1QtPy_HuIMSzhtYllT3-WeM3Sqg55wK_D'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.training')

id='1ePqnsQTJRRvQcBoF2EhoPU8CU1i5byHK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.test')

id='1ncUBWdI5AIt3FDUJokbMqpHD2knd5ebp'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.item')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [7]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-04-27 07:24:42--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.196.154.11, 3.227.65.201, 3.223.68.239, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.196.154.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13828408 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  6.69MB/s    in 2.0s    

2021-04-27 07:24:45 (6.69 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13828408/13828408]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


### Data Loading

In this Colab, we will be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/), specifically the 100K dataset (which contains in total 100,000 ratings from 1000 users on ~1700 movies).

We load the ratings data in a 80%-20% ```training```/```test``` split, while the ```items``` dataframe contains the movie titles associated to the item identifiers.

In [8]:
schema_ratings = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)])

schema_items = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("movie", StringType(), False)])

training = spark.read.option("sep", "\t").csv("MovieLens.training", header=False, schema=schema_ratings)
test = spark.read.option("sep", "\t").csv("MovieLens.test", header=False, schema=schema_ratings)
items = spark.read.option("sep", "|").csv("MovieLens.item", header=False, schema=schema_items)

In [9]:
training.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [42]:
items.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- movie: string (nullable = true)



### Your task

Let's compute some stats!  What is the number of ratings in the training and test dataset? How many movies are in our dataset?

In [15]:
print("# of ratings in the training dataset: " + str(training.count()))
print("# of ratings in the test dataset: " + str(test.count()))
print("# of movies in the dataset: " + str(items.count()))

# of ratings in the training dataset: 80000
# of ratings in the test dataset: 20000
# of movies in the dataset: 1682


Using the training set, train a model with the Alternating Least Squares method available in the Spark MLlib: [https://spark.apache.org/docs/latest/ml-collaborative-filtering.html](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

Q3


In [41]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row


# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter = 10, rank = 10, regParam = 0.1, userCol = "user_id", itemCol = "item_id", ratingCol = "rating",
          coldStartStrategy = "drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("RMSE = " + str(rmse))

RMSE = 0.9363120234990581


Q4

In [29]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als1 = ALS(maxIter = 10, rank = 100, regParam = 0.1, userCol = "user_id", itemCol = "item_id", ratingCol = "rating",
          coldStartStrategy = "drop")
model1 = als1.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions1 = model1.transform(test)
evaluator1 = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse1 = evaluator1.evaluate(predictions1)
print("RMSE = " + str(rmse1))

RMSE = 0.9296329522172235


In [110]:
regParam = [1, 0.3, 0.1, 0.03, 0.01]

rmse = []
for i in regParam:
  als_r = ALS(maxIter = 10, rank = 100, regParam = i, userCol = "user_id", itemCol = "item_id", ratingCol = "rating",
            coldStartStrategy = "drop")
  model_r = als_r.fit(training)

  predictions_r = model_r.transform(test)
  evaluator_r = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                  predictionCol="prediction")

  rmse_r = evaluator_r.evaluate(predictions_r)
  rmse.append([rmse_r, i])

print("The lowest RMSE gives regParam: ")
print(sorted(rmse, key = lambda x : x[0])[0][1])

The lowest RMSE gives regParam: 
0.1


Q6

In [104]:
# Generate the top-1 recommendation for each user in the dataset
userRecs = model1.recommendForAllUsers(1)

In [100]:
recomm = userRecs.select("recommendations")

In [138]:
def id(x):
  return x.recommendations[0].item_id

In [153]:
ID = recomm.rdd.map(lambda row : [id(row)])

In [162]:
i = ID.toDF(["id"]).toPandas()

In [166]:
i["id"].value_counts()

1449    130
318      84
313      76
50       66
64       49
       ... 
474       1
471       1
1491      1
1033      1
1024      1
Name: id, Length: 202, dtype: int64

In [192]:
movie = items.filter(items.item_id == 1449).collect()

In [193]:
movie

[Row(item_id=1449, movie='Pather Panchali (1955)')]

Once you have working code for each cell above, **head over to Gradescope, carefully read the questions, and submit your solution for this Colab**!

## Results:

*   Q2.1: 80000
*   Q2.2: 20000
*   Q2.3: 0.936
*   Q2.4: 0.930
*   Q2.5: 0.1
*   Q2.6: Pather Panchali (1955)


